<a href="https://colab.research.google.com/github/tinetor/MLOPS_FRAUD/blob/main/REST_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
#TEST DATASET

import kagglehub

# Download latest version
path = kagglehub.dataset_download("whenamancodes/fraud-detection")

print("Path to dataset files:", path)


import pandas as pd
import os

# Assuming 'path' variable holds the correct directory path
try:
    # List files in the directory to verify
    print("Files in directory:", os.listdir(path))

    # Find the CSV file within the directory
    for filename in os.listdir(path):
        if filename.endswith(".csv"):
            csv_file_path = os.path.join(path, filename)
            break
    else:
        raise FileNotFoundError("No CSV file found in the specified directory.")

    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file_path)
    print("CSV file read successfully!")
    print(df.head())

except FileNotFoundError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

Path to dataset files: /root/.cache/kagglehub/datasets/whenamancodes/fraud-detection/versions/1
Files in directory: ['creditcard.csv']
CSV file read successfully!
   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.

In [15]:
top_features = ['V14', 'V7', 'V10', 'V12']
df = df[top_features ]

In [16]:

top_features = ['V14', 'V7', 'V10', 'V12']
df = df[top_features ]


In [17]:
def create_schema_from_dataframe(df: pd.DataFrame) -> dict:
    """Creates a schema dictionary from a Pandas DataFrame, excluding the index."""
    type_mapping = {  # Mapping of Pandas dtypes to schema types
        'int64': 'integer',  # Adjust as needed
        'float64': 'number',
        'object': 'string',  # For strings or mixed types
        'bool': 'boolean',
        # Add more mappings as necessary
    }

    schema = {"fields": []}
    for col, dtype in df.dtypes.items():
        schema["fields"].append({
            "name": str(col),  # Convert column name to string
            "type": type_mapping.get(str(dtype), 'string')  # Default to string
        })
    return schema

In [18]:
# 1. Create the schema *without* the index
schema = create_schema_from_dataframe(df)

In [19]:
from pydantic import BaseModel, create_model

def create_pydantic_model_from_schema(schema: dict) -> BaseModel:
    # Mapeamento dos tipos do schema para tipos do Python
    type_mapping = {
        'integer': int,
        'number': float,
        'string': str,
        'boolean': bool
    }

    # Dicionário que armazenará os campos para o modelo Pydantic
    fields = {}
    for field in schema.get('fields', []):
        field_name = field['name']
        field_type_str = field['type']
        python_type = type_mapping.get(field_type_str, str)  # Default para str se não encontrado
        # Como todos os campos são obrigatórios, usamos '...' para indicar isso
        fields[field_name] = (python_type, ...)

    # Cria e retorna o modelo Pydantic dinâmico
    DynamicModel = create_model('DynamicModel', **fields)
    return DynamicModel

In [20]:
DynamicModel = create_pydantic_model_from_schema(schema)
import json
# Exemplo: visualizando o schema gerado
print(json.dumps(DynamicModel.model_json_schema(), indent=2))

{
  "properties": {
    "V14": {
      "title": "V14",
      "type": "number"
    },
    "V7": {
      "title": "V7",
      "type": "number"
    },
    "V10": {
      "title": "V10",
      "type": "number"
    },
    "V12": {
      "title": "V12",
      "type": "number"
    }
  },
  "required": [
    "V14",
    "V7",
    "V10",
    "V12"
  ],
  "title": "DynamicModel",
  "type": "object"
}


In [21]:
!pip install dagshub mlflow
!pip install fastapi uvicorn

In [22]:
from google.colab import userdata
#dagshub.init(repo_owner='tiago_netor', repo_name='antifraud_ml', mlflow=True)

In [23]:
import dagshub

In [26]:
import dagshub
from google.colab import userdata

# Get the token from Colab userdata
dagshub_token = userdata.get('dagshub')
# 2. Set the Dagshub token as an environment variable
os.environ['DAGSHUB_TOKEN'] = dagshub_token
# Login using the token (no interactive prompt)
dagshub.init(repo_owner='tiago_netor', repo_name='antifraud_ml', mlflow=True,)


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=f753c336-325f-46c0-a06b-6a3a61e4f870&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=1057796b3336eb3881f8f50cd77ced6befcf710117198fe8d4570caf034904f0




Accessing as tiago_netor

Initialized MLflow to track repo "tiago_netor/antifraud_ml"

Repository tiago_netor/antifraud_ml initialized!

In [39]:
import mlflow
model_uri = "models:/MyXGBoostModel@4_features_only"
model = mlflow.pyfunc.load_model(model_uri)
mlflow.pyfunc.get_model_dependencies(model_uri)



ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-12' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=SystemExit(1)>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 163, in startup
    server = await loop.create_server(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 1536, in create_server
    raise OSError(err.errno, msg) from None
OSError: [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-38-5f90304caed2>", line 69, in run

2025/02/26 12:31:42 INFO mlflow.pyfunc: To install the dependencies that were used to train the model, run the following command: '%pip install -r /tmp/tmpozxsbebj/requirements.txt'.


'/tmp/tmpozxsbebj/requirements.txt'

In [28]:
from fastapi import FastAPI
import uvicorn

In [29]:

# Mapeamento dos tipos de dados
type_mapping = {
    'integer': int,
    'number': float,
    'string': str,
    'boolean': bool
}

# Função para criar o modelo dinâmico
def create_pydantic_model_from_schema(schema: dict):
    fields = {}
    for field in schema.get('fields', []):
        field_name = field['name']
        field_type_str = field['type']
        python_type = type_mapping.get(field_type_str, str)  # Default para str
        fields[field_name] = (python_type, ...)  # '...' torna o campo obrigatório
    return create_model('DynamicModel', **fields)

# Criação da aplicação FastAPI
app = FastAPI()

# Criando o modelo dinâmico
DynamicModel = create_pydantic_model_from_schema(schema)


In [30]:
# EXTRA

In [31]:
!pip install pyngrok
from pyngrok import ngrok

In [32]:
import threading

In [33]:
from google.colab import userdata
ngrok_token=userdata.get('ngrok')

In [34]:
ngrok

<module 'pyngrok.ngrok' from '/usr/local/lib/python3.11/dist-packages/pyngrok/ngrok.py'>

In [42]:
import os
import sys
import asyncio
import threading
import nest_asyncio
import uvicorn
from fastapi import FastAPI
from pyngrok import ngrok, conf
import numpy as np
from pydantic import BaseModel, create_model
import mlflow
import pandas as pd

# Mapeamento dos tipos de dados
type_mapping = {
    'integer': int,
    'number': float,
    'string': str,
    'boolean': bool
}

# Função para criar o modelo dinâmico
def create_pydantic_model_from_schema(schema: dict):
    fields = {}
    for field in schema.get('fields', []):
        field_name = field['name']
        field_type_str = field['type']
        python_type = type_mapping.get(field_type_str, str)  # Default para str
        fields[field_name] = (python_type, ...)  # '...' torna o campo obrigatório
    return create_model('DynamicModel', **fields)

# Criação da aplicação FastAPI
app = FastAPI()

# Criando o modelo dinâmico
DynamicModel = create_pydantic_model_from_schema(schema)

@app.get("/")
async def home():
    return {"message": "API funcionando!"}
#@app.post("/predict")
#async def predict(item: DynamicModel):
#    # Preparando os dados para previsão
#    input_data = np.array([[item.Time, item.V1, item.V2]])  # Ajuste conforme o seu schema
#    prediction = model.predict(input_data)
#    return {"prediction": prediction.tolist()}

@app.post("/predict")
async def predict(item: DynamicModel):
    print("Schema:", schema)  # Print the schema
    print("Request Data:", item.dict())  # Print the request data
    try:
        data_dict = item.dict()
        input_values = list(data_dict.values())
        input_data = np.array([input_values])
        print("Input Data:", input_data)  # Print the numpy array

        prediction = model.predict(input_data)
        return {"prediction": prediction.tolist()}
    except Exception as e:
        print(f"Prediction Error: {e}")  # Print the error message
        return {"error": str(e)}  # Return the error message in the response

# Permitir que o Uvicorn rode no Colab sem conflitos de loop
nest_asyncio.apply()

# Função para rodar o servidor Uvicorn
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Rodar o servidor Uvicorn em uma thread separada
thread = threading.Thread(target=run_server, daemon=True)
thread.start()

# Autenticação do ngrok
#ngrok.set_auth_token(ngrok)  # Substitua pela sua chave
conf.get_default().auth_token = ngrok_token
ngrok.kill()
# Criar túnel com ngrok
public_url = ngrok.connect(8000).public_url
print(f"🚀 API rodando em: {public_url}")


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-20' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=SystemExit(1)>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 163, in startup
    server = await loop.create_server(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 1536, in create_server
    raise OSError(err.errno, msg) from None
OSError: [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-40-5f90304caed2>", line 69, in run

🚀 API rodando em: https://78de-34-48-79-54.ngrok-free.app


In [41]:
ngrok.kill()